jupyter notebook which goes through data cleaning 

In [1]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession, functions as F
import os
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
Picked up _JAVA_OPTIONS: -Xmx2048m
Picked up _JAVA_OPTIONS: -Xmx2048m
24/09/21 20:55:17 WARN Utils: Your hostname, Rachel resolves to a loopback address: 127.0.1.1; using 172.20.58.197 instead (on interface eth0)
24/09/21 20:55:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/21 20:55:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.parquet('../data/landing/property_data.parquet')
initial_instances = df.count()

In [3]:
import re
pdf = df.toPandas()

# Delete Rows that do not contain the price in cost_text
valid_costs = [x if re.search(r"([\d\.]+)", x) else 0 for x in pdf['cost_text']]
pdf['cost_text'] = valid_costs
pdf = pdf[pdf['cost_text']!=0]

pdf[0:100]

,url,postcode,suburb,name,cost_text,beds,baths,parking,property_type
0,https://www.domain.com.au/-leased-3-yarra-stre...,3141,south-yarra,"(Leased) 3 Yarra Street, South Yarra VIC 3141",$460,1,1,1,Apartment / Unit / Flat
1,https://www.domain.com.au/04-390-burwood-highw...,3125,burwood,"04/390 Burwood Highway, Burwood VIC 3125","$310 per week, with AC",1,1,0,Apartment / Unit / Flat
2,https://www.domain.com.au/1-2-32-folkstone-cre...,3156,ferntree-gully,"1 & 2/32 Folkstone Crescent, Ferntree Gully VI...",$800,4,3,1,House
3,https://www.domain.com.au/1-acacia-street-torq...,3228,torquay,"1 Acacia Street, Torquay VIC 3228","$1,000 per week",4,2,2,House
4,https://www.domain.com.au/1-aleppo-crescent-fr...,3200,frankston-north,"1 Aleppo Crescent, Frankston North VIC 3200",$450,3,1,1,House
...,...,...,...,...,...,...,...,...,...
97,https://www.domain.com.au/1-17-view-street-pas...,3044,pascoe-vale,"1/17 View Street, Pascoe Vale VIC 3044",$690pw / $2998pcm,3,2,1,Townhouse
98,https://www.domain.com.au/1-177-widford-street...,3047,broadmeadows,"1/177 Widford Street, Broadmeadows VIC 3047",$550 Per Week,3,2,2,House
99,https://www.domain.com.au/1-18-arnott-street-c...,3168,clayton,"1/18 Arnott Street, Clayton VIC 3168",$280 per week,1,1,1,House
100,https://www.domain.com.au/1-18-arthur-street-s...,3141,south-yarra,"1/18 Arthur Street, South Yarra VIC 3141",$750pw,2,2,1,Apartment / Unit / Flat


In [4]:
sdf = spark.createDataFrame(pdf)

In [5]:
raw_sdf = sdf.withColumn(
    # Properties priced per week
    'Week',
    F.when(F.lower(F.col('cost_text')).contains('pw') \
           | F.lower(F.col('cost_text')).contains('p/w') \
           | F.lower(F.col('cost_text')).contains('week'), True).otherwise(False)
).withColumn(
    # see if property is priced per annum
    'month',
    F.when(F.lower(F.col('cost_text')).contains('month') \
        | F.lower(F.col('cost_text')).contains('per month') \
        | F.lower(F.col('cost_text')).contains('pcm'), True ).otherwise(False)
).withColumn(
    # remove $ and ,
    'cost_pw',
    F.regexp_replace(F.col("cost_text"), r"[$,]", "")
).withColumn(
    # see if property is priced per annum
    'contains_pa',
    F.when(F.lower(F.col('cost_text')).contains('p.a') \
        | F.lower(F.col('cost_text')).contains('pa') \
        | F.lower(F.col('cost_text')).contains('per annum') \
        | F.lower(F.col('cost_text')).contains('per_annum') \
        | F.lower(F.col('cost_text')).contains('p/a'), True ).otherwise(False)
).withColumn(
    # removes decimal places
    'cost_text',
    F.regexp_replace(F.col("cost_text"), r"\.\d+", "")
).withColumn(
    # grab price
    'cost_pw',
    F.regexp_extract(F.col("cost_text"), r"(\$*\d+(?:,\d*)*(?:\.\d*)?)", 1)
).withColumn(
    # if property is priced per annum, divide by 52
    'cost_pw',
    F.when(F.col('contains_pa') == True, F.round(F.col('cost_text') / 52, 2)).otherwise(F.col('cost_text'))
).withColumn(
    # if property is priced per month, divide by 4
    'cost_pw',
    F.when(F.col('month') == True, F.round(F.col('cost_text') / 4, 2)).otherwise(F.col('cost_text'))
).withColumn(
    # see if property is furnished - note: this will contain partially furnished as well
    'furnished', 
    F.when(F.lower(F.col('cost_text')).contains('furnish') \
        | F.lower(F.col('cost_text')).contains('furniture'), True ).otherwise(False)
).drop('contains_pa').withColumn(
    # remove $ and ,
    'cost_pw',
    F.regexp_replace(F.col("cost_pw"), r"[$,\D]", "") # casting to float makes it include the decimal point
)

# Returns Null in cost_pw when there's multiple prices

In [6]:
# pdf = raw_sdf.toPandas()
# pdf[pdf['month']==True]


In [7]:
raw_sdf

url,postcode,suburb,name,cost_text,beds,baths,parking,property_type,Week,month,cost_pw,furnished
https://www.domai...,3141,south-yarra,(Leased) 3 Yarra ...,$460,1,1,1,Apartment / Unit ...,false,false,460,false
https://www.domai...,3125,burwood,04/390 Burwood Hi...,"$310 per week, wi...",1,1,0,Apartment / Unit ...,true,false,310,false
https://www.domai...,3156,ferntree-gully,1 & 2/32 Folkston...,$800,4,3,1,House,false,false,800,false
https://www.domai...,3228,torquay,"1 Acacia Street, ...","$1,000 per week",4,2,2,House,true,false,1000,false
https://www.domai...,3200,frankston-north,1 Aleppo Crescent...,$450,3,1,1,House,false,false,450,false
https://www.domai...,3011,footscray,1 Bed 1 Bath/34 C...,$480 P/W,1,1,0,Apartment / Unit ...,true,false,480,false
https://www.domai...,3011,footscray,1 Bed 1 Bath/48 C...,$500 P/W,1,1,1,Apartment / Unit ...,true,false,500,false
https://www.domai...,3030,point-cook,1 Bensonhurst Par...,$570 per week,4,2,2,House,true,false,570,false
https://www.domai...,3047,broadmeadows,"1 Biltris Court, ...",$500,3,1,2,House,false,false,500,false
https://www.domai...,3149,mount-waverley,1 Birralee Street...,$625 per week,3,1,1,House,true,false,625,false


In [8]:
import warnings
warnings.filterwarnings("ignore")

# This code resolves issues relating to multiple prices being listed
# Extracts the weekly cost
pdf = raw_sdf.toPandas()
row = -1
for x in pdf['cost_text']: 
    row += 1
    prices = re.findall(r"(\$*\d+,?\d+)", x)
    if (len(prices)>1):
        pdf['cost_pw'].iloc[row] = prices[0] # taking lower price when a range is listed

In [9]:
# Shouldn't have None values!!
pdf[pdf['cost_pw'].isna()==True] 
# property type car space??

,url,postcode,suburb,name,cost_text,beds,baths,parking,property_type,Week,month,cost_pw,furnished
108,https://www.domain.com.au/1-1a-civic-avenue-ec...,3564,echuca,"1/1a Civic Avenue, Echuca VIC 3564",$2275 calendar month,3,2,3,House,False,True,None,False
316,https://www.domain.com.au/101-154-chapel-stree...,3182,st-kilda,"101/154 Chapel Street, St Kilda VIC 3182",Stay from 1 month - $1162pw,2,2,1,Apartment / Unit / Flat,True,True,None,False
318,https://www.domain.com.au/101-288-tooronga-roa...,3146,glen-iris,"101/288 Tooronga Road, Glen Iris VIC 3146",Stay from 1 month - $850pw,1,1,1,Apartment / Unit / Flat,True,True,None,False
320,https://www.domain.com.au/101-466-dandenong-ro...,3161,caulfield-junction,"101/466 Dandenong Road, Caulfield North VIC 3161",Stay from 1 month - $990pw,2,2,1,Apartment / Unit / Flat,True,True,None,False
608,https://www.domain.com.au/120-127-mc-kenzie-st...,3995,wonthaggi,"120-127 Mc Kenzie Street, Wonthaggi VIC 3995",$77 per calendar month,1,1,0,Apartment / Unit / Flat,False,True,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11236,https://www.domain.com.au/68-hayward-lane-melb...,3000,melbourne,"68 Hayward Lane, Melbourne VIC 3000",$150 per month,1,1,1,House,False,True,None,False
11766,https://www.domain.com.au/9-9-campaspe-street-...,3564,echuca,"9/9 Campaspe Street, Echuca VIC 3564",$2687 calendar month,2,2,1,Apartment / Unit / Flat,False,True,None,False
11855,https://www.domain.com.au/carpark-403-135-abec...,3000,melbourne,"Carpark 403/135 Abeckett Street, Melbourne VIC...",$250/month,0,1,1,Carspace,False,True,None,False
11858,https://www.domain.com.au/carpark-80-abeckett-...,3000,melbourne,"Carpark/80 Abeckett Street, Melbourne VIC 3000",$250 per month,0,1,1,Carspace,False,True,None,False


In [10]:
# Convert pdf back to spark
raw_sdf = spark.createDataFrame(pdf)
raw_sdf

url,postcode,suburb,name,cost_text,beds,baths,parking,property_type,Week,month,cost_pw,furnished
https://www.domai...,3141,south-yarra,(Leased) 3 Yarra ...,$460,1,1,1,Apartment / Unit ...,false,false,460,false
https://www.domai...,3125,burwood,04/390 Burwood Hi...,"$310 per week, wi...",1,1,0,Apartment / Unit ...,true,false,310,false
https://www.domai...,3156,ferntree-gully,1 & 2/32 Folkston...,$800,4,3,1,House,false,false,800,false
https://www.domai...,3228,torquay,"1 Acacia Street, ...","$1,000 per week",4,2,2,House,true,false,1000,false
https://www.domai...,3200,frankston-north,1 Aleppo Crescent...,$450,3,1,1,House,false,false,450,false
https://www.domai...,3011,footscray,1 Bed 1 Bath/34 C...,$480 P/W,1,1,0,Apartment / Unit ...,true,false,480,false
https://www.domai...,3011,footscray,1 Bed 1 Bath/48 C...,$500 P/W,1,1,1,Apartment / Unit ...,true,false,500,false
https://www.domai...,3030,point-cook,1 Bensonhurst Par...,$570 per week,4,2,2,House,true,false,570,false
https://www.domai...,3047,broadmeadows,"1 Biltris Court, ...",$500,3,1,2,House,false,false,500,false
https://www.domai...,3149,mount-waverley,1 Birralee Street...,$625 per week,3,1,1,House,true,false,625,false
